# Ordenando un toque:
    - Lo que buscamos es encontrar algún (o algunos) modelos que nos permitan aproximar alguna de las columnas del dataset (si, alguna).
    - Si o si tenemos que hacerlo co nla columna "precio", pero además tenemos que elgir otra (está en el enunciado).
    
    - Separemos los tantos:
        - Feature Engeniere es algo que vamos a hacer POR FUERA de todo lo que es la ejecución, es decir, cuando nosotros hagamos fit + predict + loquesea, esta nueva columna no tiene idea de que es nueva, asi que esto no nos importa tanto (para lo que es la ejecucion pura y dura)
       
        - Segmentación + K-fold + errores, es algo que nos interesa DENTRO de la ejecución.
        - Ejecutar 1 vez y tomar metricas, no estaría bueno, por lo que ya sabemos (overfitting), entonces no nos queda otra más que para cada segmento, realizar el KFold, callcular las métricas en cada Fold, y guardar el promedio de esas métricas (nos queda un array de metricas promedio en cada segmento).
        - ¿Donde empieza el prolema? En que para cada Fold, los valores de los factores de la ecuación de la recta (esos que obtenemos cuando hacemos fit) van a cambiar siempre, porque para cada "cacho" de dataset, estas cambian.
        - ¿Nos importa realmente? Obviamente googlie esto porque estaba medio perdido, y la respuesta es que no (a priori). Lo primero que a nosotros nos interesa saber, es si nuestro modelo "esta bueno", es decir, si el error total es bajo (comparando distintas métricas y demas). Esto no nos da las variables "finales", una vez que definimos que algún modelo funciona bien, recién ahí ejecutamos la regresión con TODOS los datos, y no quedamos con los factores resultantes (si usamos segmentación, seria un conjunto de factores para cada segmento, y para predecír deberíamos saber determinar a que segmento pertenece)
        
        
Esto lo saque de acá:
https://stats.stackexchange.com/questions/52274/how-to-choose-a-predictive-model-after-k-fold-cross-validation
https://stats.stackexchange.com/questions/2306/feature-selection-for-final-model-when-performing-cross-validation-in-machine?rq=1

Por si estoy entendiendo cualquier cosa.

In [2]:
import metnum
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error as RMSE, mean_squared_log_error as RMSLE

/home/diego/.pyenv/versions/3.6.5/envs/metnum-tp3/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
df_original = pd.read_csv('../data/train.csv')
df = df_original.copy()

In [201]:
class Segment:
    def __init__(self, name, linear_regressor, features):
        self.name = name
        self.linear_regressor = linear_regressor
        self.features = features
        
    def fit(self, A, reales):
        self.reales = reales
        self.linear_regressor.fit(A, self.reales)

    def predict(self, A):
        return self.linear_regressor.predict(A)
    
    def execute(self, A, reales, metrics, k=5):

        self.errors = np.zeros_like(metrics)

        kf = KFold(n_splits=k, shuffle=True)
        for train_index, test_index in kf.split(A):
            # Divido en datos de entrenamiento y testeo
            A_train, A_test = A[train_index], A[test_index]
            reales_train, reales_test = reales[train_index], reales[test_index]
            # Fiteo con los de entrenamiento
            self.linear_regressor.fit(A_train, reales_train)

            # Predigo con los de testeo
            aproximados = self.linear_regressor.predict(A_test)
            i = 0
            # Calculo y guardo las metricas pasadas por parametro
            for metric in metrics:
                self.errors[i] += metric(reales_test, aproximados)
                i += 1
        
        self.errors /= k


In [202]:
def regresionPorSegmento(df, segment_column, features, metrics, predict_column='precio'):
    segments = np.array([])
    
    for name in df[segment_column].unique():
        # Leo los datos correspondientes al segmeto
        df_segment = df[df[segment_column] == name] # Aquellos datos que no contengan la columna, no son tomados en cuenta
        
        A = df_segment[features].values
        precios_reales = df_segment[predict_column].values

        # Creo el segmento
        segment = Segment(name, metnum.LinearRegression(), features)
        #print(f'Segemento: {name} \n')
        # Fit y predict
        segment.execute(A, precios_reales, metrics)
        
        # Guardo el segmento
        segments = np.append(segments, segment)

    return segments

In [218]:
def exec_model(df, segment, features, metrics):
    # Limpia los valores con NAN que se vayan a utilizar
    df = df.dropna(subset=(features + [segment]))
    
    return regresionPorSegmento(df, segment, features, metrics)

In [219]:
# Modelo fruta
df = df_original.copy()
segment = 'provincia'
features = ['metrostotales', "metroscubiertos"]   #'metrostotales', 'metroscubiertos', "habitaciones", "banos", "antiguedad"]

metrics = [RMSE]#[RMSE, RMSLE]

segments = exec_model(df, segment, features, metrics)

In [222]:
for s in segments:
    print(s.errors)

[4156107885240.9775]
[1354412708844.659]
[1830182298885.3223]
[774634830650.0808]
[1431666412229.9932]
[388640914289.60803]
[407897160163.955]
[2506749316086.738]
[933856933240.9371]
[459247932698.7167]
[551484402926.4608]
[3092034460481.793]
[1150638477418.2314]
[956793462736.651]
[467064320918.2663]
[660033938342.2181]
[449915174219.1372]
[885650975027.6184]
[508837846973.6808]
[427616119754.3148]
[242556008114.54095]
[330958216504.785]
[154959573190.17496]
[584660761617.0188]
[781720470843.069]
[230215917367.29623]
[334821962388.4232]
[440352351412.7943]
[771014841369.2185]
[221647262635.2679]
[1254908871418.5251]
[482228833088.6156]
